In [1]:
import os
filename = 'Abyssinian_1.jpg'
extension = os.path.splitext(filename)[1]
label = filename.replace(extension, '')
print(label)

Abyssinian_1


In [2]:
import os
path_input = os.path.join('data/pets/images')
list_filename = os.listdir(path_input)

list_filenames = os.listdir(path_input)
list_file = []
for filename in list_filenames:
    if ('newfoundland' in filename) or ('pomeranian' in filename):
        label = 0 # dog
    elif ('Abyssinian' in filename) or ('Bombay' in filename):
        label = 1 # cat
    else:
        continue
    list_file.append([filename, label, filename.split('_')[0]])
print(list_file[0])
# ['Abyssinian_1.jpg', 1, 'Abyssinian']

['pomeranian_39.jpg', 0, 'pomeranian']


In [3]:
from sklearn.model_selection import train_test_split

random_seed = 0

list_train, list_val = train_test_split(list_file, shuffle=True, random_state=random_seed, test_size=0.2)
list_val, list_test = train_test_split(list_val, shuffle=True, random_state=random_seed, test_size=0.5)

In [4]:
import os
from PIL import Image
import torch.utils.data as data

class MyDataset(data.Dataset):
    def __init__(self, list_file, transform=None, phase='train'):
        self.list_file = list_file
        self.transform = transform
        self.phase = phase

    def __len__(self):
        # ファイル数を返す
        return len(self.list_file)

    def __getitem__(self, index):
        # 画像をPillowsで開く
        path_input = './data/pets/images/'
        path_image = os.path.join(path_input, self.list_file[index][0])
        pil_image = Image.open(path_image)

        # 画像の前処理
        image_transformed = self.transform(pil_image).convert('RGB')

        # ラベルを取得
        label_class = self.list_file[index][1]
        label_type = self.list_file[index][2]
        return image_transformed, label_class

In [5]:
from torchvision import transforms

class ImageTransform():
    def __init__(self, resize, mean, std):
        self.data_transform = transforms.Compose([
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.ToTensor(),
            transforms.Normalize(mean, std)
        ])

    def __call__(self, image):
        return self.data_transform(image)

In [6]:
resize = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
transform = ImageTransform(resize, mean, std)

In [8]:
# dataset
train_dataset = MyDataset(list_train, transform=transform, phase='train')
val_dataset = MyDataset(list_val, transform=transform, phase='val')

In [10]:
# dataloader
batch_size = 8

train_dataloader = data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [15]:
# デバイスを選択
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [14]:
from torchvision import models
net = models.vgg16(weights='VGG16_Weights.IMAGENET1K_V1')
# 出力層を2つに付け替える

import torch.nn as nn
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

In [16]:
import torch
print(torch.version.cuda)

12.8
